# RL

The goal of this notebook is to introduce an RL agent into EdgeSimPy. 

How:
- Pass the rl agent to the simulator through the instantiation: 
```python
simulator = esp.Simulator(tick_duration=1,tick_unit="seconds",
    stopping_criterion=stop_on_n_steps,
    resource_management_algorithm=RL,
    scheduler=esp.activation_schedulers.ChaosScheduler,
)
```

Since I'll be building this with the Ray library, I'll need a new environment. 

I have one named `Python (edgesimpy)`

This environment will not have edgesimpy in it's pip list or conda list. 

This environment will be for edge chaos. 

Just to say it, I named the environment I used before edgesympy_tut, but passed the name `Python (edgesimpy)` to the jupyter kernel. That is a little confusing and I vote we never do it again. 



# Define the environment as a class

Implement a custom environment class that encapsulates the dynamics of your network simulation. This class should provide methods to interact with the environment, such as reset() to reset the environment to its initial state and step(action) to take a step based on the given action.

In [1]:
import edge_sim_py as esp
        
def place_and_migrate_to_least_utilized(parameters):
    for service in esp.Service.all():
        if not service.being_provisioned:
            edge_servers = sorted(
                esp.EdgeServer.all(), key=lambda s: (
                    (s.cpu - s.cpu_demand) 
                   * (s.memory - s.memory_demand) 
                   * (s.disk - s.disk_demand)
                  ) ** (1/3),
                reverse=True,
            )
            for edge_server in edge_servers: # use ordered list
                if edge_server.has_capacity_to_host(service=service):
                    if service.server != edge_server:
                        # print(f"[STEP {parameters['current_step']}] Migrating {service} From {service.server} to {edge_server}")
                        service.provision(target_server=edge_server)
                        break

def stop_on_n_steps(model: object):    
    return model.schedule.steps == 30

simulator = esp.Simulator(tick_duration=1,tick_unit="seconds",
    stopping_criterion=stop_on_n_steps,
    resource_management_algorithm=place_and_migrate_to_least_utilized,
    scheduler=esp.activation_schedulers.ChaosScheduler,
)
simulator.initialize(input_file="../edgesimpy-tutorials/datasets/sample_dataset2.json")


In [2]:
import ray
from ray import tune
#from ray.rllib.agents import ppo # old 
from ray.rllib.algorithms import ppo

# Define your RL agent class
class MyRLAgent:
    def __init__(self, config):
        # Initialize your RL agent with any necessary parameters
        pass
    
    def learn(self, state, action, reward, next_state):
        # Update your RL agent's policy based on observed experience
        pass

# Define your resource management algorithm function
def resource_management_algorithm(parameters):
    # Extract relevant information from the parameters
    state = parameters["state"]
    
    # Use the RL agent to make decisions based on the current state
    action = rl_agent.decide(state)
    
    # Return the action to be taken
    return action

# Initialize Ray
ray.init()

# Set up the RL training environment
config = {
    # Define any necessary configurations for your RL agent
}

# Register your custom RL agent class with Ray RLlib
tune.register_trainable("my_agent", MyRLAgent)

# Train your RL agent using Ray RLlib
analysis = tune.run(
    "PPO",
    config=config,
    stop={"training_iteration": 100},
    checkpoint_at_end=True
)

# Retrieve the trained RL agent
trained_agent = ppo.PPOTrainer(config=config, env="your_environment")

# Use the trained agent for inference or further evaluation


2024-03-14 19:10:50,857	INFO worker.py:1724 -- Started a local Ray instance.


TypeError: ('Second argument must be convertable to Trainable', <class '__main__.MyRLAgent'>)

Ok.... Thinking through a bit 


Ideally I am a math expert who can help ya with mathy things. I've seen no evidence that people are looking for this skill, but it where I think I should invest. 

The project I have chosen, edge chaos, involves
- edgeSimPy, which I'd say I have now learned to an intermediate level 
- reinforcement learning, which I have learned at a intro level. 
- Ray, which I am quite confused about. 

I think that the right place to pu my focus next is Ray. It seems to have this distributed learning and RL things built in, and it seems like I will practice some python things I have not had a chance to practice before. 


That said, jumping ship is bad. 
- edge literature I've been researching... how much do I have left? Is it a finite project? 
- will I get going with ray and forget about edgesimpy? or can I have edgesimpy in mind as I learn Ray? I'll shoot for the latter of course. 
- ISL.... Aakash is out there, and willing to talk. That reading group could grow, and it is a great way to dwell on these ML basics that I never really learned. 

I also have this idea that I could take some time to review statistics,
- Koosis, wasserman, etc... I think this would take a solid week of full time work. 

The idea is tomake this like a BOAT. I would take a training story at this point, I think. 

I have no notes on ray in my github... I must have lost it in the layoff... 

